<a href="https://colab.research.google.com/github/Claudia060392/Portafolio-Ciencia-de-Datos/blob/main/Practicas%20Consolidadas/Modulo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

# Subir el archivo manualmente
archivo = files.upload()

# Carga del CSV
df = pd.read_csv("migracion.csv")

print(df.head())

Saving migracion.csv to migracion.csv
  Pais_Origen Pais_Destino   Año  Cantidad_Migrantes Razon_Migracion  \
0   Venezuela       España  2018              250000       Económica   
1       Siria     Alemania  2015              800000       Conflicto   
2      México         EEUU  2020              300000       Económica   
3       India       Canadá  2019              150000       Educativa   
4  Bangladesh       Italia  2021              120000       Económica   

   PIB_Origen  PIB_Destino  IDH_Origen  IDH_Destino  
0        5000        30000        0.70         0.89  
1        2000        45000        0.55         0.94  
2        9000        65000        0.78         0.92  
3        2500        48000        0.65         0.93  
4        2000        35000        0.61         0.89  


In [2]:
# Módulo 3 - Práctica Consolidada
# Análisis de datos de migración con NumPy y Pandas

import numpy as np
import pandas as pd

# -------------------------------
# Funciones auxiliares
# -------------------------------
def limpiar_columnastring(df, columna, reemplazo="sin informacion"):
    df[columna] = df[columna].astype("string")
    df[columna] = df[columna].fillna(reemplazo)
    return df

# -------------------------------
# 1) Carga y limpieza básica
# -------------------------------
df = pd.read_csv("migracion.csv")
print("Primeras filas:")
print(df.head())

print("\nValores perdidos por columna:")
print(df.isnull().sum())

# Conversión de fecha si existe una columna de fechas
for posible_fecha in ["Fecha", "fecha", "fechas", "FECHA"]:
    if posible_fecha in df.columns:
        df[posible_fecha] = pd.to_datetime(df[posible_fecha], errors="coerce")
        break

# Imputación simple
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mean())
    else:
        if df[col].isnull().any():
            df = limpiar_columnastring(df, col, reemplazo="sin informacion")

print("\nNulos luego de imputación:")
print(df.isnull().sum())

# Normalización de etiquetas
if "Razon_Migracion" in df.columns:
    df["Razon_Migracion"] = df["Razon_Migracion"].astype("string")
    df["Razon_Migracion"] = df["Razon_Migracion"].replace({
        "Económica": "Trabajo", "Economica": "Trabajo",
        "Conflicto": "Guerra",
        "Educación": "Estudios", "Educacion": "Estudios"
    })

# -------------------------------
# 2) Outliers con IQR
# -------------------------------
# Reconocer una columna de cantidad de migrantes
posibles_migrantes = ["Migrantes", "Cantidad_Migrantes", "N_Migrantes", "Migrantes_Total", "migrantes"]
col_migrantes = None
for nombre in df.columns:
    if nombre in posibles_migrantes or nombre.lower() in ["migrantes", "cantidad_migrantes"]:
        col_migrantes = nombre
        break
if col_migrantes is None:
    # fallback simple: primera columna numérica
    numericas = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    col_migrantes = numericas[0] if len(numericas) > 0 else None

if col_migrantes is not None:
    q1 = df[col_migrantes].quantile(0.25)
    q3 = df[col_migrantes].quantile(0.75)
    iqr = q3 - q1
    inferior = q1 - 1.5 * iqr
    superior = q3 + 1.5 * iqr

    outl = df[(df[col_migrantes] < inferior) | (df[col_migrantes] > superior)]
    df = df[(df[col_migrantes] >= inferior) & (df[col_migrantes] <= superior)]

    print("\nIQR aplicado en:", col_migrantes)
    print("q1:", q1, "q3:", q3, "iqr:", iqr)
    print("Límite inferior:", inferior, "Límite superior:", superior)
    print("Registros removidos por outliers:", len(outl))
else:
    print("\n[Aviso] No se detectó columna de migrantes. Se continúa sin IQR.")

# -------------------------------
# 3) Análisis exploratorio (EDA)
# -------------------------------
print("\n.info():")
print(df.info())
print("\n.describe():")
print(df.describe(include="all"))

# Algunas métricas
candidatos_pib_origen  = ["PIB_Origen", "PIB_origen", "PIBOrigen"]
candidatos_pib_destino = ["PIB_Destino", "PIB_destino", "PIBDestino"]
candidatos_idh_origen  = ["IDH_Origen", "IDH_origen", "IDHOrigen"]
candidatos_idh_destino = ["IDH_Destino", "IDH_destino", "IDHDestino"]

def pick(colnames):
    for c in colnames:
        if c in df.columns:
            return c
    return None

col_pib_origen  = pick(candidatos_pib_origen)
col_pib_destino = pick(candidatos_pib_destino)
col_idh_origen  = pick(candidatos_idh_origen)
col_idh_destino = pick(candidatos_idh_destino)

if col_migrantes is not None:
    print(f"\nMedia de {col_migrantes}:", df[col_migrantes].mean())
    print(f"Mediana de {col_migrantes}:", df[col_migrantes].median())

if col_pib_origen is not None:
    print("PIB promedio (origen):", df[col_pib_origen].mean())
if col_pib_destino is not None:
    print("PIB promedio (destino):", df[col_pib_destino].mean())

if "Razon_Migracion" in df.columns:
    print("\nConteo por razón de migración:")
    print(df["Razon_Migracion"].value_counts())

# -------------------------------
# 4) Agrupamiento y ordenar
# -------------------------------
if "Razon_Migracion" in df.columns and col_migrantes is not None:
    print("\nSuma de migrantes por razón:")
    print(df.groupby("Razon_Migracion")[col_migrantes].sum())

if "Razon_Migracion" in df.columns and col_idh_origen is not None:
    print("\nMedia de IDH_Origen por razón:")
    print(df.groupby("Razon_Migracion")[col_idh_origen].mean())

if col_migrantes is not None:
    df = df.sort_values(by=col_migrantes, ascending=False)
    print("\nOrdenado por cantidad de migrantes (desc):")
    print(df.head())

# -------------------------------
# 5) Filtros y columna derivada
# -------------------------------
# Filtrar conflictos
if "Razon_Migracion" in df.columns:
    conflicto = df[df["Razon_Migracion"].isin(["Guerra", "Conflicto"])]
    print("\nMigraciones por conflicto (primeras filas):")
    print(conflicto.head())

# IDH destino > 0.90
if col_idh_destino is not None:
    altos = df[df[col_idh_destino] > 0.90]
    print(f"\nFilas con {col_idh_destino} > 0.90 (primeras filas):")
    print(altos.head())

# Diferencia_IDH = Origen - Destino
if (col_idh_origen is not None) and (col_idh_destino is not None):
    df["Diferencia_IDH"] = df[col_idh_origen] - df[col_idh_destino]
    print("\n'Diferencia_IDH' creada (primeras filas):")
    print(df[["Diferencia_IDH"]].head())

# -------------------------------
# 6) Exportación
# -------------------------------
df.to_csv("Migracion_Limpio.csv", index=False)
print("\nArchivo 'Migracion_Limpio.csv' guardado sin índice.")

from google.colab import files
files.download("Migracion_Limpio.csv")


Primeras filas:
  Pais_Origen Pais_Destino   Año  Cantidad_Migrantes Razon_Migracion  \
0   Venezuela       España  2018              250000       Económica   
1       Siria     Alemania  2015              800000       Conflicto   
2      México         EEUU  2020              300000       Económica   
3       India       Canadá  2019              150000       Educativa   
4  Bangladesh       Italia  2021              120000       Económica   

   PIB_Origen  PIB_Destino  IDH_Origen  IDH_Destino  
0        5000        30000        0.70         0.89  
1        2000        45000        0.55         0.94  
2        9000        65000        0.78         0.92  
3        2500        48000        0.65         0.93  
4        2000        35000        0.61         0.89  

Valores perdidos por columna:
Pais_Origen           0
Pais_Destino          0
Año                   0
Cantidad_Migrantes    0
Razon_Migracion       0
PIB_Origen            0
PIB_Destino           0
IDH_Origen            0
IDH_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>